# Visualisation Project

In [1]:
#pip install geopy

In [2]:
# Import all required modules
import pandas as pd
#import random
import numpy as np
#import pprint
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
import math

import os
import time # Used for playing around with timers. 
from sys import getsizeof # Used for checking how much memory a variable is using. 
import geopy.distance

## GTFS - Static

##### Import the GTFS - static data and take a look at it

First download the static transport data and save locally.
https://opendata.transport.nsw.gov.au/dataset/public-transport-timetables-realtime-v2

In [3]:
# The GTFS-static data is already available to us in the (hidden) 'gtfs_static_2020' folder
# If we start with '/' we go up to the top of the directory folder
pathname = '/course/data/gtfs_static_2020'

# Let's see what we have available in the (hidden) 'gtfs_static_2020' folder

# For every file that is in the list in the directory print out the name of the file
for file in os.listdir(pathname):
    print(file, ";", str(os.path.getsize(pathname+'/'+file)/1000)+' kb')
# Even thoough the data set is hidden we can still access it
# At the end we have the size of the file. The default unit is 'bytes'



FileNotFoundError: [Errno 2] No such file or directory: '/course/data/gtfs_static_2020'

In [ ]:
agencydf = pd.read_csv(pathname+'/'+'agency.txt')
#agencydf.head(3)


In [ ]:
tripsdf = pd.read_csv(pathname+'/'+'trips.txt')
#tripsdf.head(3)

In [ ]:
caldf = pd.read_csv(pathname+'/'+'calendar.txt')
#caldf.head(3)

In [ ]:
cal_datesdf = pd.read_csv(pathname+'/'+'calendar_dates.txt')
#cal_datesdf.head(3)

In [ ]:
stopsdf = pd.read_csv(pathname+'/'+'stops.txt')
#stopsdf.head(3)

In [ ]:
stop_timesdf = pd.read_csv(pathname+'/'+'stop_times.txt')
#stop_timesdf.head(3)

In [ ]:
routesdf = pd.read_csv(pathname+'/'+'routes.txt')
#routesdf.head(3)

# Make a list of routes that are part of the Sydney Buses Network-- ditching trains, ferry, light rail. There is too much data to import all trips (even just 15 seconds of it)
busroutes = routesdf[routesdf['route_desc']=='Sydney Buses Network']['route_id'].tolist()


In [ ]:
# For visualisation purposes
routesdf[routesdf['route_desc']=='Sydney Buses Network']

In [ ]:
tripsdf = pd.read_csv(pathname+'/'+'trips.txt')
tripsdf.head(3)

# Narrow it down to only trips belonging to bus routes. 
tripsdf=tripsdf[tripsdf['route_id'].isin(busroutes)]

In [ ]:
# NOT ALL THE BUS ROUTES IN TRIPS.TXT ACTUALLY HAVE STOP TIMES!
# Therefore, by starting with the stops.txt many routes get lost.

#df =routesdf[routesdf['route_desc'] == 'Sydney Buses Network']
#list1
#routesdf[routesdf['route_desc'] == 'Sydney Buses Network']
#df['route_id_num'] = df['route_id'].map(lambda x: str(x).split('-')[1])
#list1 = df['route_id_num'].tolist()
#list1

#tripsdf['route_id_num'] = tripsdf['route_id'].map(lambda x: str(x).split('-')[1])
#tripsdf[tripsdf['route_id_num'].str.contains('B1')]
#list3=tripsdf[tripsdf['route_id_num'].str.contains('B1')]['trip_id'].tolist()

#stop_timesdf[stop_timesdf['trip_id'].isin(list3)]
#tripsdf[tripsdf['route_id_num'].str.contains('B1')]

##### Get all bus routes in sydney buses network      -    Update to include only buses that we're interested in

In [ ]:
# Get stop ids for stops on military road (not the military road in Watsons bay or in Dover Heights)
# There are many military roads. It's simpler to just get the main stops along this strip rather than get all the stops
# and sort through them all.
#stop_id_neutBayJunct = stopsdf[(stopsdf['stop_name'].str.contains('Warringah Mall')) & (stopsdf['stop_name'].str.contains('Neutral')) & (stopsdf['stop_name'].str.contains('Junction'))]['stop_id'].tolist()
###stop_id_central = stopsdf[(stopsdf['stop_name'].str.contains('Central')) & (stopsdf['stop_name'].str.contains('Station'))]['stop_id'].tolist()
#stop_id_central

##stopsdf[(stopsdf['stop_name'].str.contains('Military')) & (stopsdf['stop_name'].str.contains('Dover')==False) & (stopsdf['stop_name'].str.contains('Watsons')==False)].sort_values('stop_lon',ascending=True)['stop_name'].tolist()
#stopsdf[(stopsdf['stop_name'].str.contains('Central')) & (stopsdf['stop_name'].str.contains('Station'))&(stopsdf['stop_name'].str.contains('Stand'))]

In [ ]:
# Get the trip ids for all those trips that stop at Central using stop_times.txt
###trips_central = stop_timesdf[stop_timesdf['stop_id'].isin(stop_id_central)]['trip_id'].tolist()
###len(trips_central)
# Note that this includes all agencies at this stage. We will only start talking about buses when we get to the routesdf

In [ ]:
# Get all unique route ids for the trips that use central road using the trips.txt
###routeids_central = tripsdf[tripsdf['trip_id'].isin(trips_central)]['route_id']
#routeids_central

In [ ]:
# Get all of the routes in the Sydney Bus Network (SBN)
routesdf_SBN = routesdf[routesdf['route_desc'] == 'Sydney Buses Network']
#routesdf_SBN

routeIdLIST_SBN = routesdf_SBN['route_id'].astype(str).values.tolist()
#routeIdLIST_SBN

# Only consider trips that are for routes in the SBN
tripsdf_SBN = tripsdf[tripsdf.route_id.isin(routeIdLIST_SBN)]
#tripsdf_SBN

In [ ]:
# Filter regular services
# We will first convert start and end date to datetime format.
# Regular trips run from 20200306 to 20200604


caldf['start_dt']=caldf['start_date'].map(lambda x: datetime.strptime(str(x),"%Y%m%d"))
caldf['end_dt']=caldf['end_date'].map(lambda x: datetime.strptime(str(x),"%Y%m%d"))
caldf['relevance_interval']= caldf['end_dt'] - caldf['start_dt']+pd.Timedelta('1 days')
caldf = caldf.sort_values('relevance_interval',ascending=False)

# Want only rows in relevance interval that have values corresponding to the maximmum relevance intervals
caldf = caldf[caldf['relevance_interval']==caldf['relevance_interval'].max()]

# Consider only buses that run every weekday
#caldf['day count'] = caldf[['monday','tuesday','wednesday','thursday','friday']].sum(axis=1)
#caldf = caldf[caldf['day count']==caldf['day count'].max()]

service_regular = caldf['service_id'].tolist()

caldf.head(3)

In [ ]:
# Get only the 'buses' that are on the 'route's from above AND 'run regularly'
tripsdf_SBN = tripsdf_SBN[tripsdf_SBN['service_id'].isin(service_regular)]

''' Clean up tripsdf - get rid of unnecessary columns
columns_list = tripsdf.columns.tolist()
if columns_list.count('bikes_allowed') | columns_list.count('trip_note'):
    columns_list.remove('bikes_allowed')
    columns_list.remove('trip_note')
tripsdf=tripsdf[columns_list]'''

# Clean up route_id representation
#tripsdf['route_id_num'] = tripsdf['route_id'].map(lambda x: str(x).split('-')[1])

len(tripsdf_SBN)

In [ ]:
# Get a list of all the trips for the 320 and 430 routes.
#triplist = tripsdf['trip_id'].tolist()

# Count the number of daily weekday trips that stop at central and run throughout the entire study period
#central_trip_static_count = len(triplist)

In [ ]:
# Get triplist as updated by changes in tripsdf_SBN
triplist = tripsdf_SBN.trip_id.astype(str).values.tolist()
len(triplist)

## GTFS - Real-time

In [ ]:
#pip install --upgrade gtfs-realtime-bindings

In [ ]:
# Import packages required for working with real-time data
from google.transit import gtfs_realtime_pb2 # library for GTFS-r bindings
import requests # For making requests over the internet. If you give it a url this is how you get the info back

### Trip Updates

##### Import Data

In [ ]:
# Set up the call to the API
# Step 1, define a dictionary called headers that includes your api key
# You should use your own API key. Put it where the Xs are
headers =  {'Authorization': 'apikey XXXXXXXXXXXXXXXXXXXXXXXXXXXX'}
# We are creating a dictionary called 'headers', key is 'Authorization' and the string is the apikey.
# Step 2, initialise the FeedMessage class in the library
feed = gtfs_realtime_pb2.FeedMessage()

# Classes are things that have known attributes
# eg. the datetime has attributes like year month date, etc...
# Classes are things where you know it's qualities , FILL IN
# Here we're initialising the feed ... FILL IN

# Step 3, Get the data from the API URL
response_TU = requests.get('https://api.transport.nsw.gov.au/v1/gtfs/realtime/buses', headers=headers,allow_redirects=True)
# Response is whatever you get when you call the URL. i.e. when you call the url you get content back.


In [ ]:
# A short aside about not crashing your workspace:

# Gives the memory used by that variable in bytes. 
getsizeof(response_TU)

# Memory is when your computer needs to be able to access infromation for computations
# e.g. a variable is stored in memory. When you restart the kernel you clear the memory
# Each workspace is allocated a certain amount of memory -- set up by ed workspace

In [ ]:
#Step 4, assuming the URL request was successful....
t1 = datetime.now()
if response_TU.status_code == 200:   # 200 is a standard response code for 'It worked!'
    feed.ParseFromString(response_TU.content) # pass the data to the FeedMessage
    entitylist = [entity for entity in feed.entity if entity.HasField('trip_update')] # Make a list of the messages
t2 = datetime.now()
print(t2-t1)



In [ ]:
# Grabbing the content and parsing it yields a much larger variable. As we will see below, it's also human-readable (sort of)
getsizeof(entitylist)
# It got a lot bigger because we took the data and 'unpacked it'

In [ ]:
# Every time you run this, you should see a different number of messages. 
#There will be more during peak periods and more when there are lot of delays

# Each trip update is a different entity -- it changes depending on the time. (It is bigger at 8am versus 3am)
len(entitylist)


In [ ]:
# What are these messages? They are parsed protocol buffers. 
# A single message contains many updates pertaining to one trip_id
entitylist[0]

# Ways of very efficiently packing and storing large amounts of data
# Do that by not syaing what the data is -- just passes an ordered list of data
# you have to know that the first things is something, second thing is another thing, etc.
# This makes it a lot more compact
# To use it you need other info like a 'key' or a 'protocal'

# This is not a dictionary, but it's a bit like one with keys and values.

# Every trip update corresponds to exactly 1 trip id but contains multiple stop ids

In [ ]:
# Grab only the updates that belong to a set of trips

t1 = datetime.now()
# Create a blank dataframe
df_TU = pd.DataFrame(columns=['trip_id','route_id','stop_id','stop_seq','start_time','start_date','arr_time','arr_delay','dep_time','dep_delay'])
# Loop through your list of messages.  
for entity in entitylist:
    trip_id = entity.trip_update.trip.trip_id   # Each message pertains to a single trip ID
    
    if trip_id in triplist: # Filter by the trip IDs you have stored in your list.
        
        # Define a set of variables that you want to keep. 
        start_time = entity.trip_update.trip.start_time # The time that trip started
        start_date = entity.trip_update.trip.start_date # The date that trip started
        route_id = entity.trip_update.trip.route_id # The route ID of the trip.
        
        # NOTE: route ids are different in GTFS realtime and GTFS static
        # They've identified the bus by the number but also the contract setup to run that bus
        # Contract numbers can repeat a lot because of many services that are on the same contract.
            
        for stupdate in entity.trip_update.stop_time_update: # Any trip update will have several stop updates 
            
            if (stupdate.arrival.delay!=0) | (stupdate.departure.delay != 0): # If you only want those updates that actually have a non-zero delay. 
                
                # Define a set of variables that you want to keep. 
                adelay = stupdate.arrival.delay # How much delay at arrival
                ddelay = stupdate.departure.delay # How much delay at departure
                stop_seq = stupdate.stop_sequence # What sequence the stop ID is along the route
                atime = stupdate.arrival.time # The time at arrival
                dtime = stupdate.departure.time # The time at departure
                stop_id = stupdate.stop_id # The stop ID associated with the update
                # Add all this information into a new dataframe that is 1 row long
                df2_TU = pd.DataFrame({'trip_id':trip_id,'route_id':route_id,'stop_id':stop_id,'stop_seq':stop_seq,'start_time':start_time,'start_date':start_date,'arr_time':atime,'arr_delay':adelay,'dep_time':dtime,'dep_delay':ddelay},index=[0]) # Putting index =[0] helps pandas understand this single line ____
                
                df_TU = pd.concat([df_TU,df2_TU],ignore_index=True) # And stick it on the bottom of the existing dataframe
                # We dont want to worry about the indexing; we write ignore_index=True which will allow us to keep adding rows indefinitely
                
        t2 = datetime.now()
        #print(t2-t1)   # Print the elapsed time for each trip_id.
        t1=t2

In [ ]:
# Take a look at what we ended up with
df_TU

In [ ]:
# Save the realtime data:
df_TU.to_csv('20200507_1705_TripUpdates_SBN_ALL.csv')

#df_TU.sort_values('dep_delay', ascending = False)

##### Working with realtime Trip Updates:

In [ ]:
'''# Get the number trips that there are updates for. This is for the Figure 2 plot.
unique_trip_id_list = df_TU['trip_id'].unique().tolist()
central_trip_RT_count = len(unique_trip_id_list)'''

In [ ]:
'''
# What is the distribution of delay observed in these updates?
plt.hist(df_TU['arr_delay'],bins = list(range(-800,200,100)))
plt.xlabel('Delay at arrival (s)')
plt.ylabel('Number of trip updates')
plt.show()
#plt.savefig('distribution_of_delay.png')
'''

In [ ]:
'''
# How many different routes are covered by those trips AND how many updates are available for all of the buses on the route
df_TU[['route_id','trip_id']].groupby('route_id').count()
'''

### Vehicle Positions

In [ ]:
# Set up the call to the API
# Step 1, define a dictionary called headers that includes your api key
headers =  {'Authorization': 'apikey XXXXXXXXXXXXXXXXXXXXXXXXXXXX'}
# Step 2, initialise the FeedMessage class in the library
feed = gtfs_realtime_pb2.FeedMessage()
# Step 3, Get the data from the API URL
response_VP = requests.get('https://api.transport.nsw.gov.au/v1/gtfs/vehiclepos/buses', headers=headers,allow_redirects=True)


In [ ]:
# A short aside about not crashing your workspace:

# Gives the memory used by that variable in bytes. 
getsizeof(response_VP)

In [ ]:
#Step 4, assuming the URL request was successful....
t1 = datetime.now()
if response_VP.status_code == 200:
    feed.ParseFromString(response_VP.content) # pass the data to the FeedMessage
    entitylist = [entity for entity in feed.entity ] #if entity.HasField('trip_update')] # Make a list of the messages
t2 = datetime.now()
print(t2-t1)

In [ ]:
# What are these messages? They are parsed protocol buffers. 
# A single message contains many updates pertaining to one trip_id
#entitylist[0]

In [ ]:
# Grab only the updates that belong to a set of trips

t1 = datetime.now()
# Create a blank dataframe
df_VP = pd.DataFrame(columns=['trip_id','route_id','start_time','start_date','lat','lon','bearing','speed','occupancy','congestion'])
# Loop through your list of messages.  
for entity in entitylist:
    trip_id = entity.vehicle.trip.trip_id # Each message pertains to a single trip ID
    if trip_id in triplist: # Filter by the trip IDs you have stored in your list. 
        # Define a set of variables that you want to keep. 
        start_time = entity.vehicle.trip.start_time # The time that trip started
        start_date = entity.vehicle.trip.start_date # The date that trip started
        route_id = entity.vehicle.trip.route_id # The route ID of the trip.
        if entity.vehicle.congestion_level!='RUNNING_SMOOTHLY': # If you only want the positions of buses that are delayed.
                # Define a set of variables that you want to keep. 
                lat = entity.vehicle.position.latitude # latitude
                lon = entity.vehicle.position.longitude # longitude
                bearing = entity.vehicle.position.bearing # which way the bus is facing
                speed = entity.vehicle.position.speed # Speed
                occupancy = entity.vehicle.occupancy_status # How many passengers are onboard (levels)
                congestion = entity.vehicle.congestion_level # whether traffic is flowing smoothly. 

                # Add all this information into a new dataframe that is 1 row long
                df2_VP =pd.DataFrame({'trip_id':trip_id,'route_id':route_id,'start_time':start_time,'start_date':start_date,'lat':lat,'lon':lon,'bearing':bearing,'speed':speed,'occupancy':occupancy,'congestion':congestion},index=[0])
                # And stick it on the bottom of the existing dataframe
                df_VP = pd.concat([df_VP,df2_VP],ignore_index=True)
        t2 = datetime.now()
        #print(t2-t1)   # Print the elapsed time for each trip_id.
        t1=t2

In [ ]:
# Take a look at what you ended up with:
df_VP.head()

In [ ]:
# Save the realtime data
df_VP.to_csv('20200507_1705_VehiclePositions_SBN_ALL.csv')

In [43]:
# Take a look at what you ended up with:
df_VP['route_id_num'] = df_VP['route_id'].map(lambda x: str(x).split('_')[1])


In [44]:
# Merge df_VP with the tripsdf using the key of route_id
# The resulting dataframe should only contain trip ids that are currently in service
# i.e. merge on inner
df_VP = pd.merge(df_VP, tripsdf[['trip_id','direction_id',]], on='trip_id', how = 'inner')
df_VP.head()
#df300_VP.sort_values(['bearing', 'lon', 'lat'],ascending = True)

,trip_id,route_id,start_time,start_date,lat,lon,bearing,speed,occupancy,congestion,route_id_num,direction_id
0,872360,2434_854,17:30:00,20200504,-33.923683,150.927521,15.0,0.0,1,0,854,0
1,247942,2434_873,17:45:00,20200504,-34.019455,150.865799,77.0,3.8,1,0,873,0
2,946973,2434_858,17:50:00,20200504,-34.007423,150.739212,178.0,10.3,1,0,858,0
3,1049674,2435_T80,16:55:00,20200504,-33.851425,150.896286,169.0,0.0,1,1,T80,0
4,871871,2434_872,16:45:00,20200504,-34.014565,150.860092,13.0,15.9,1,1,872,1


In [45]:
#df_VP[df_VP['trip_id'] == '1127944']['route_id'].values.tolist()


In [46]:
rt_trip_id_list = df_VP['trip_id'].tolist()
rt_trip_id_list


['872360',
 '247942',
 '946973',
 '1049674',
 '871871',
 '1097916',
 '1123921',
 '1050290',
 '944657',
 '871356',
 '871735',
 '849594',
 '893662',
 '947033',
 '905858',
 '1018914',
 '1050688',
 '1019484',
 '1049957',
 '42033',
 '1123897',
 '871907',
 '1019503',
 '1049924',
 '1049958',
 '1118750',
 '961878',
 '672079',
 '1141481',
 '871870',
 '1123945',
 '1074842',
 '1049276',
 '1049641',
 '1050893',
 '872061',
 '1144308',
 '753602',
 '613030',
 '1050600',
 '944726',
 '871960',
 '446357',
 '1049836',
 '774223',
 '880368',
 '247938',
 '180118',
 '1050824',
 '1050882',
 '471391',
 '1050438',
 '966658',
 '672146',
 '180119',
 '1144305',
 '1049926',
 '1049860',
 '866515',
 '1123899',
 '561260',
 '1050776',
 '1049235',
 '1049275',
 '944460',
 '1049257',
 '1019546',
 '944593',
 '871460',
 '1049895',
 '944817',
 '1035618',
 '871965',
 '1049580',
 '780570',
 '774309',
 '780515',
 '1144346',
 '1050185',
 '1049242',
 '1035924',
 '1117380',
 '1050332',
 '944818',
 '606606',
 '35971',
 '1050562',
 

In [47]:

#df_VP = df_VP.sort_values(['route_id_num','direction_id'],ascending=True)
min_distance_values = []
min_distance_tripIds = []
otherTripIds = []
rt_trip_id_list = df_VP['trip_id'].tolist()

# Go through each trip_id
for tripId in rt_trip_id_list:
    
    del(otherTripIds)
    otherTripIds = rt_trip_id_list.copy()
    otherTripIds.remove(tripId)
    
    # Initialise min_distance with a rediculously large number (1000 km). This value needs to reset on each iteration
    min_distance = float(1000*1000)
    min_distance_trip_id='None'
    
    # See which trip id (for trips going in the same direction and on the same route) corresponds the closest bus to the current trip
    for otherIDs in otherTripIds:
        
        # If the route ids are the same and the trips are both going in the same direction then calculate the distance between them
        if (df_VP[df_VP['trip_id'] == tripId]['route_id'].values.tolist() == df_VP[df_VP['trip_id'] == otherIDs]['route_id'].values.tolist()) & (df_VP[df_VP['trip_id'] == tripId]['direction_id'].values.tolist() == df_VP[df_VP['trip_id'] == otherIDs]['direction_id'].values.tolist()):
            
            coords_1 = df_VP[df_VP['trip_id'] == tripId][['lat','lon']].values
            coords_2 = df_VP[df_VP['trip_id'] == otherIDs][['lat','lon']].values
            distance_bw_trips = geopy.distance.geodesic(coords_1, coords_2).km*1000
            
            #distance_bw_trips = ((df_VP[df_VP['trip_id'] == tripId]['lat'].values - df_VP[df_VP['trip_id'] == otherIDs]['lat'].values)**2+(df_VP[df_VP['trip_id'] == tripId]['lon'].values - df_VP[df_VP['trip_id'] == otherIDs]['lon'].values)**2)**(1/2)
            
        
            if min_distance > distance_bw_trips:
                min_distance = distance_bw_trips
                min_distance_trip_id = otherIDs
                
    
    
    
    
    min_distance_values.append(min_distance)
    min_distance_tripIds.append(min_distance_trip_id)

    
    
df_VP_dummy =pd.DataFrame({'trip_id':rt_trip_id_list,'Distace bus infront/behind (m)':min_distance_values,'trip_id_close_bus':min_distance_tripIds})
df_VP = pd.merge(df_VP,df_VP_dummy, on = 'trip_id')



df400_VP = df_VP[df_VP['route_id_num'].str.startswith('4')]
#df300_VP = df_VP[df_VP['route_id_num'].str.startswith('3')]

#df300_VP
df400_VP

#df400_VP_dir0
#df400_VP_dir1
#df300_VP_dir0
#df300_VP_dir1

#df_VP

KeyboardInterrupt: 

In [ ]:
list(range(0,2000,100))

In [ ]:
# What is the distribution of delay observed in these updates?
#NOTE: this caption is wrong, it should say "Distance to nearest bus (m)"
plt.hist(df400_VP['Distace bus infront/behind (m)'],bins = list(range(0,2500,200)))
plt.xlabel('Distance to nearest bus (m)')
plt.ylabel('Number of trip updates')
plt.show()
#plt.savefig('distribution_of_delay.png')

In [ ]:
#df400_VP.sort_values('Distace nearest bus infront/behind (m)').head()

### Figure 1 - Subplot with 2 histograms

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2)


#ax1.plot(x, y)
#ax2.plot(x, -y)



ax1.hist(df_TU['arr_delay'],bins = list(range(-800,200,50)),color ='black')
ax1.set_xlabel('Delay at arrival (s)',fontsize=15)
ax1.set_ylabel('Number of trip updates',fontsize=15)
ax1.set_position([0,0.3, 0.8, 0.8])
ax1.set_title('A',loc='left')

ax2.hist(df400_VP['Distace bus infront/behind (m)'],bins = list(range(0,2500,150)),color ='black')
ax2.set_xlabel('Distance to bus in front (m)',fontsize=15)
ax2.set_ylabel('Number of trip updates',fontsize=15)
ax2.set_position([1,0.3, 0.8, 0.8])
ax2.set_title('B',loc='left')
ax2.locator_params(axis='y', integer=True)


txt="""Fig. 1: The relationship between buses that are delayed and the distance between successive buses. 
      (A) The delay at arrival for the period that was analised shows a bi-modal distribution with most buses making early arrivals.
      (B) The bar representing the largest distance infront of a bus corresponds to a large headway.
      The trip that is experiencing this large headway is likely bunching with the trip behind it. 
      This means, while (A) shows that buses are running early, (B) shows evidence that bus bunching is occuring nonetheless.
"""

plt.figtext(0, -0.20, txt, wrap=False, horizontalalignment='left', fontsize=15)

#plt.tight_layout()


# Save the figure
plt.savefig('Figure1.png',bbox_inches='tight')

plt.show()


### Save Real-time Data

### Plot Figure 2 - Bar Chart

In [ ]:
# Bar chart
# Set the figure size/aspect ratio and the resolution. 
fig, ax = plt.subplots(figsize=(7,5))#figsize=(4.5,4.5), dpi=200)
# 'fig' is a variable to represent the figure, 
# 'ax' is a variable to represent the axes --> dpi = dots per inch (it's the resolution that you specify)
# Need to put above statement at the start of plots, esp. when they are being printed

# For categorical data bar chart is almost always the better solution bw pie and bar
plt.bar(['GTFS-static','GTFS-realtime'], [central_trip_static_count,central_trip_RT_count], width=0.5 , align='center',color = 'black')
#width => change the width of the bars
plt.xlabel('Data Sources',fontsize=15)
plt.ylabel('Number of Observations',fontsize=15)

txt="""Fig. 2: This chart shows the quantity of trip data collected for bus routes which have regular services and intersect the 430
      bus route. At this stage in the study there is more static data available than realtime data, however since real-time data 
      updates every 15 seconds the amount of data that can be sourced is large compared to the GTFS-static data set. Static 
      trip information is only valuble in this study if there is real time data. Thus, it is important to collect real-time trip data regularly."""
plt.xticks(fontsize=14)
plt.figtext(0, -0.18, txt, wrap=False, horizontalalignment='left', fontsize=15)
plt.tight_layout()

# Save the figure
plt.savefig('Figure2.png',bbox_inches='tight')
# when saving you need to specifiy the file name AND the extension

plt.show()